## Evaluate a model from its best checkpoint

In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
import os
import wandb

from model.metrics import Metrics
from model.main import CNN
from utils.helpers import get_config

os.environ["WANDB__SERVICE_WAIT"] = "300"

#### Set the config file to train the model

In [4]:
# Specify the model to evaluate by passing the corresponding config file from the configs/ directory
CONFIG = "configs/sparsebagnet_cox.yml"
LOG_WANDB = True

# Set the split(s) to evaluate on (a list of "val" and/or "test")
splits = ["test"] 

#### Run evaluation

In [7]:
c = get_config(CONFIG)
PROJECT = c.cnn.project
RUN_ID = c.cnn.run_id

# Evaluate from best checkpoint
c.cnn.load_best_model = True
c.cnn.resume_training = False

estimator = Metrics(c)
cnn = CNN(c=c, estimator=estimator)

# Ignore the config entry "eval_sets" and evaluate on the splits defined above
for split in splits:
    print("\nEvaluating on: ", split)

    cnn.evaluate(split=split, log_all = LOG_WANDB)
   
    print(cnn.estimator.get_all_performances())

# Finish the run, otherwise it might log into the last run when re-running the script
wandb.finish()